### Example 2: Constraining the total capacity per bus and carrier

In [1]:
import pypsa
import pandas as pd
import highspy


In [2]:

network = pypsa.examples.ac_dc_meshed(from_master=True)


INFO:pypsa.io:Imported network ac-dc-meshed.nc has buses, carriers, generators, global_constraints, lines, links, loads


In [3]:
pd.Series(network.snapshots)

0   2015-01-01 00:00:00
1   2015-01-01 01:00:00
2   2015-01-01 02:00:00
3   2015-01-01 03:00:00
4   2015-01-01 04:00:00
5   2015-01-01 05:00:00
6   2015-01-01 06:00:00
7   2015-01-01 07:00:00
8   2015-01-01 08:00:00
9   2015-01-01 09:00:00
Name: snapshot, dtype: datetime64[ns]

In [4]:

network.links_t.p_set = pd.DataFrame(index=network.snapshots)


In [5]:
network.buses

,v_nom,carrier,x,y,type,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,generator,sub_network
Bus,,,,,,,,,,,,
London,380.0,AC,-0.13,51.50,,,1.0,0.0,inf,PQ,,
Norwich,380.0,AC,1.30,52.60,,,1.0,0.0,inf,PQ,,
Norwich DC,200.0,DC,1.30,52.50,,,1.0,0.0,inf,PQ,,
Manchester,380.0,AC,-2.20,53.47,,,1.0,0.0,inf,PQ,,
Bremen,380.0,AC,8.80,53.08,,,1.0,0.0,inf,PQ,,
Bremen DC,200.0,DC,8.80,52.98,,,1.0,0.0,inf,PQ,,
Frankfurt,380.0,AC,8.70,50.12,,,1.0,0.0,inf,PQ,,
Norway,380.0,AC,10.75,60.00,,,1.0,0.0,inf,PQ,,
Norway DC,200.0,DC,10.75,60.00,,,1.0,0.0,inf,PQ,,


In [6]:

network.add(
    "Generator",
    "Frankfurt Wind 2",
    bus="Frankfurt",
    capital_cost=120,
    carrier="wind",
    p_nom_extendable=True,
)



In [7]:
network.generators

,bus,capital_cost,efficiency,marginal_cost,p_nom,p_nom_extendable,p_nom_min,carrier,control,type,...,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,weight,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
Manchester Wind,Manchester,2793.651603,1.000000,0.110000,80.0,True,100.0,wind,PQ,,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
Manchester Gas,Manchester,196.615168,0.350026,4.532368,50000.0,True,0.0,gas,PQ,,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
Norway Wind,Norway,2184.374796,1.000000,0.090000,100.0,True,100.0,wind,PQ,,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
Norway Gas,Norway,158.251250,0.356836,5.892845,20000.0,True,0.0,gas,PQ,,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
Frankfurt Wind,Frankfurt,2129.456122,1.000000,0.100000,110.0,True,100.0,wind,PQ,,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
Frankfurt Gas,Frankfurt,102.676953,0.351666,4.086322,80000.0,True,0.0,gas,PQ,,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
Frankfurt Wind 2,Frankfurt,120.000000,1.000000,0.000000,0.0,True,0.0,wind,PQ,,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0


In [8]:
type(network.buses)

pandas.core.frame.DataFrame

In [9]:
network.buses.loc[["Frankfurt", "Manchester"], "nom_min_wind"] = 2000
network.buses.loc[["Frankfurt"], "nom_max_wind"] = 2200
network.buses.loc["Frankfurt"]

v_nom            380.0
carrier             AC
x                  8.7
y                50.12
type                  
unit                  
v_mag_pu_set       1.0
v_mag_pu_min       0.0
v_mag_pu_max       inf
control             PQ
generator             
sub_network           
nom_min_wind    2000.0
nom_max_wind    2200.0
Name: Frankfurt, dtype: object

In [10]:
network.optimize(solver_name="highs")

Index(['2', '3', '4'], dtype='object', name='Line')
Index(['0', '1', '5', '6'], dtype='object', name='Line')
Index(['2', '3', '4'], dtype='object', name='Line')
Index(['0', '1', '5', '6'], dtype='object', name='Line')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.03s
INFO:linopy.solvers:Log file at /private/var/folders/m5/570n50js4fb24y9ht7nd_pm00000gn/T/highs.log.
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 199 primals, 492 duals
Objective: -1.38e+07
Solver model: available
Solver message: optimal

/Users/miratheidel/Documents/Programmierung/Projects/pypsa_experiments/pypsa_venv/lib/python3.11/site-packages/pypsa/optimization/optimize.py:357: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are

Running HiGHS 1.5.3 [date: 2023-05-16, git hash: 594fa5a9d]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
403 rows, 198 cols, 964 nonzeros
402 rows, 198 cols, 962 nonzeros
Presolve : Reductions: rows 402(-90); columns 198(-1); elements 962(-91)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.5896414189e+07 Pr: 61(162995); Du: 0(5.47058e-11) 0s
        219    -1.3793078250e+07 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 219
Objective value     : -1.3793078250e+07
HiGHS run time      :          0.01


('ok', 'optimal')

In [11]:
import linopy
linopy.available_solvers

['highs']

In [12]:
network.generators

,bus,capital_cost,efficiency,marginal_cost,p_nom,p_nom_extendable,p_nom_min,carrier,control,type,...,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,weight,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
Manchester Wind,Manchester,2793.651603,1.000000,0.110000,80.0,True,100.0,wind,Slack,,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,2000.000000
Manchester Gas,Manchester,196.615168,0.350026,4.532368,50000.0,True,0.0,gas,PQ,,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,-0.000000
Norway Wind,Norway,2184.374796,1.000000,0.090000,100.0,True,100.0,wind,PQ,,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,895.372552
Norway Gas,Norway,158.251250,0.356836,5.892845,20000.0,True,0.0,gas,PQ,,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,91.001539
Frankfurt Wind,Frankfurt,2129.456122,1.000000,0.100000,110.0,True,100.0,wind,Slack,,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,100.000000
Frankfurt Gas,Frankfurt,102.676953,0.351666,4.086322,80000.0,True,0.0,gas,PQ,,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,884.092697
Frankfurt Wind 2,Frankfurt,120.000000,1.000000,0.000000,0.0,True,0.0,wind,PQ,,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,2100.000000


In [13]:
network.generators.p_nom_opt

Generator
Manchester Wind     2000.000000
Manchester Gas        -0.000000
Norway Wind          895.372552
Norway Gas            91.001539
Frankfurt Wind       100.000000
Frankfurt Gas        884.092697
Frankfurt Wind 2    2100.000000
Name: p_nom_opt, dtype: float64